## Examining Student and Online Perceptions of LSE

### Part 1: Introduction

Before arriving at LSE, many students likely hear of the university's low student satisfaction rates. These impressions often gain traction on online forums such as Reddit, where anecdotal accounts tend to amplify perceptions of dissatisfaction. Additionally, multiple university ranking websites also present LSE as having a comparatively low student satisfaction rate. Therefore, it is worth exploring this topic, to understand how "severe" the problem is and whether the data fully aligns with the online perception of the university.

We can explore the statement above through a series of focused questions:
- How does LSE student satisfaction compare to other universities?
- How does student satisfaction within LSE vary by degree?
- How has LSE student satisfaction evolved over the last few years (2020-2023)?
- How is LSE perceived by people online?

It is worth considering what is meant by student satisfaction. Many data sources such as the National Student Survey break down student's views into categories such as teaching, course content, course organisation and assessment. Meanwhile, online the focus tends to be on social opportunities, as well as societies. As "student satisfaction" is multifaceted we will be breaking down each question into the specific type of student satisfaction we are discussing, to avoid confusion. The NSS is generally better suited towards 'academic' related definitions of student satisfaction, such as student's views on the quality of the course, the teaching etc. whilst Reddit and online platforms likely better represent student's thoughts on their social lives at certain universities.




### Part 2: Data acquisition
- Go to 'Data_acquisition.ipynb' file

### Part 3: Data Preparation and Exploration

In [2]:
import pandas as pd
import pickle

### NSS data

We can answer many of the questions through the NSS. The NSS questions students on multiple different categories, such as teaching quality, course quality and assessment quality. Most websites ignore this distinction and simply focus on one of these metrics such as teaching quality, however, we wanted to make the distinction between categories clearer. As there are multiple categories, we create multiple dataframes so that visualisation is easier. Another important thing to consider is the response to population size; if it is high it means the measure of student satisfaction is more likely to accurately represent the real student body, compared to if it is low, when the results are less robust. This is the reason why many online displays of these metrics leave out universities like the University of Oxford, where a hefty proportion of the students boycott the NSS.

We also wanted to display the variance between student's opinions on these measures through the standard deviation score.

In [2]:
file_path = './data/univ_df.pkl'

# Load the dictionary from the pickle file
with open(file_path, 'rb') as pickle_file:
    univ_df = pickle.load(pickle_file)

#refresher of the universities surveyed
for key in univ_df.keys():
    print(key)

LSE
Oxford
UCL
Birmingham
Edinburgh
Glasgow
Imperial
KCL
Manchester
Norwich
Strathclyde
Warwick


Below, we create dataframes representing the differences in student opinions on teaching quality, course quality, assessment quality, support quality, as well as course organisation quality. We considered aggregating every metric into one 'overall satisfaction' score, however, feel that would not take into account the fact that some may view certain criterion as deserving more weight; for example, course quality may be valued much more than assessment quality. 

The way each criterion's score has been calculated is through averaging 'Positivity Measures' to groups of questions. For example, the 'Teaching Quality' dataframe takes the following questions into account:

- Q01: How good are teaching staff at explaining things?
- Q02: How often do teaching staff make the subject engaging?

The Course Quality measures the following:

- Q03: How often is the course intellectually stimulating?
- Q04: How often does your course challenge you to achieve your best work?
- Q05: To what extent have you had the chance to explore ideas and concepts in depth?
- Q06: How well does your course introduce subjects and skills in a way that builds on what you have already learned?
- Q07: To what extent have you had the chance to bring together information and ideas from different topics?
- Q08: To what extent does your course have the right balance of directed and independent study?
- Q09: How well has your course developed your knowledge and skills that you think you will need for your future?

*Course Quality differs from Course Organisation in that Course Quality focuses more on the knowledge conveyed in the course, rather than the resources and overall organisation of the course*

Assessment Quality measures the following questions:

- Q10: How clear were the marking criteria used to assess your work?
- Q11: How fair has the marking and assessment been on your course?
- Q12: How well have assessments allowed you to demonstrate what you have learned?
- Q13: How often have you received assessment feedback on time?

Support:

- Q14: How often does feedback help you to improve your work?
- Q15: How easy was it to contact teaching staff when you needed to?
- Q16: How well have teaching staff supported your learning?

Course Organisation:

- Q17: How well organised is your course?
- Q18: How well were any changes to teaching on your course communicated?
- Q19: How well have the IT resources and facilities supported your learning?
- Q20: How well have the library resources (e.g., books, online services and learning spaces) supported your learning?
- Q21: How easy is it to access subject specific resources (e.g., equipment, facilities, software) when you need them?

Please remember that these categories represent student's *opinions* about the categories themselves and are not an objective measure of what a university's facilities are actually like; for instance, students at universities with high ranks may expect more from said university. 



In [62]:
teach_dict={}
course_dict={}
assess_dict={}
support_dict={}
organisation_dict={}

for name,uni_df in univ_df.items():
    #We measure 'All Subjects' at this point as we don't want to differentiate by Subject
    teach_df=uni_df[uni_df["Subject level"]=='All subjects'][:2]
    pos_average=teach_df['Positivity measure (%)'].sum()/2
    pos_average=round(pos_average,2)
    sd_average=teach_df['Standard deviation'].sum()/2
    response_ratio=teach_df['Responses'].sum()/teach_df['Population'].sum()
    response_ratio=round(response_ratio, 2)
    teach_dict[name]={'Positivity Measure(%)':pos_average, 'Standard Deviation': sd_average, 'Response Ratio': response_ratio}

    course_df=uni_df[uni_df["Subject level"]=='All subjects'][2:9]
    pos_average= course_df['Positivity measure (%)'].sum()/course_df['Positivity measure (%)'].count() #7 rows
    pos_average=round(pos_average,2)
    sd_average= course_df['Standard deviation'].sum()/course_df['Standard deviation'].count()
    sd_average=round(sd_average,2)
    response_ratio=course_df['Responses'].sum()/course_df['Population'].sum()
    response_ratio=round(response_ratio, 2)
    course_dict[name]={'Positivity Measure(%)':pos_average, 'Standard Deviation': sd_average, 'Response Ratio': response_ratio}
    
    assess_df=uni_df[uni_df["Subject level"]=='All subjects'][9:13]
    pos_average=assess_df['Positivity measure (%)'].sum()/assess_df['Positivity measure (%)'].count()
    pos_average=round(pos_average,2)
    sd_average= assess_df['Standard deviation'].sum()/assess_df['Standard deviation'].count()
    sd_average=round(sd_average,2)
    response_ratio=assess_df['Responses'].sum()/assess_df['Population'].sum()
    response_ratio=round(response_ratio, 2)
    assess_dict[name]={'Positivity Measure(%)':pos_average, 'Standard Deviation': sd_average, 'Response Ratio': response_ratio}
    
    support_df=uni_df[uni_df["Subject level"]=='All subjects'][13:16]
    pos_average=support_df['Positivity measure (%)'].sum()/support_df['Positivity measure (%)'].count()
    pos_average=round(pos_average,2)
    sd_average=support_df['Standard deviation'].sum()/support_df['Standard deviation'].count()
    sd_average=round(sd_average,2)
    response_ratio=support_df['Responses'].sum()/support_df['Population'].sum()
    response_ratio=round(response_ratio, 2)
    support_dict[name]={'Positivity Measure(%)':pos_average, 'Standard Deviation': sd_average, 'Response Ratio': response_ratio}
    
    org_df=uni_df[uni_df["Subject level"]=='All subjects'][16:21]
    pos_average=org_df['Positivity measure (%)'].sum()/org_df['Positivity measure (%)'].count()
    pos_average=round(pos_average,2)
    sd_average=org_df['Standard deviation'].sum()/org_df['Standard deviation'].count()
    sd_average=round(sd_average,2)
    response_ratio=org_df['Responses'].sum()/org_df['Population'].sum()
    response_ratio=round(response_ratio, 2)
    organisation_dict[name]={'Positivity Measure(%)':pos_average, 'Standard Deviation': sd_average, 'Response Ratio': response_ratio}
    
    
    

In [63]:
teach_df=pd.DataFrame(teach_dict)
print('Student Views on Teaching Quality') #a higher percentage means more students think positively of that factor
teach_df.head()

Student Views on Teaching Quality


,LSE,Oxford,UCL,Birmingham,Edinburgh,Glasgow,Imperial,KCL,Manchester,Norwich,Strathclyde,Warwick
Positivity Measure(%),84.70,91.45,83.40,82.90,84.80,85.5,86.10,83.40,81.00,85.30,87.95,86.80
Standard Deviation,0.95,0.75,0.50,0.50,0.60,0.6,0.75,0.55,0.45,1.35,0.65,0.50
Response Ratio,0.67,0.50,0.72,0.69,0.65,0.7,0.72,0.70,0.74,0.82,0.74,0.72


In [41]:
course_df=pd.DataFrame(course_dict)
print('Student Views on Course Quality')
course_df.head()

Student Views on Course Quality


,LSE,Oxford,UCL,Birmingham,Edinburgh,Glasgow,Imperial,KCL,Manchester,Norwich,Strathclyde,Warwick
Positivity Measure(%),81.94,85.73,81.14,81.06,78.36,82.21,87.23,79.34,78.59,79.31,85.26,84.69
Standard Deviation,1.03,0.87,0.53,0.57,0.66,0.67,0.79,0.60,0.50,1.53,0.77,0.60
Response Ratio,0.67,0.50,0.71,0.69,0.65,0.70,0.72,0.70,0.74,0.82,0.74,0.72


In [42]:
print('Student Views on Assessment Quality')
assess_df=pd.DataFrame(assess_dict)
assess_df.head()

Student Views on Assessment Quality


,LSE,Oxford,UCL,Birmingham,Edinburgh,Glasgow,Imperial,KCL,Manchester,Norwich,Strathclyde,Warwick
Positivity Measure(%),71.57,72.38,70.88,71.00,65.38,71.78,71.35,69.95,69.95,82.22,75.05,78.72
Standard Deviation,1.12,1.00,0.60,0.62,0.75,0.75,0.92,0.65,0.52,1.55,0.85,0.62
Response Ratio,0.67,0.49,0.71,0.69,0.64,0.70,0.72,0.70,0.74,0.82,0.74,0.72


In [43]:
print('Student Views on Academic Support Quality')
support_df=pd.DataFrame(support_dict)
support_df.head()

Student Views on Academic Support Quality


,LSE,Oxford,UCL,Birmingham,Edinburgh,Glasgow,Imperial,KCL,Manchester,Norwich,Strathclyde,Warwick
Positivity Measure(%),81.47,87.50,75.93,73.47,72.90,75.50,77.17,73.87,74.23,84.70,80.77,81.07
Standard Deviation,1.03,0.87,0.53,0.63,0.67,0.73,0.87,0.63,0.53,1.50,0.77,0.60
Response Ratio,0.67,0.50,0.71,0.69,0.64,0.70,0.72,0.70,0.74,0.82,0.74,0.71


In [45]:
print('Student Views on Course Organisation Quality')
organisation_df=pd.DataFrame(organisation_dict)
organisation_df.head()

Student Views on Course Organisation Quality


,LSE,Oxford,UCL,Birmingham,Edinburgh,Glasgow,Imperial,KCL,Manchester,Norwich,Strathclyde,Warwick
Positivity Measure(%),84.66,82.44,83.36,81.48,79.00,79.44,84.26,77.82,77.26,78.74,83.88,86.20
Standard Deviation,1.00,0.90,0.52,0.54,0.64,0.68,0.76,0.62,0.52,1.56,0.76,0.58
Response Ratio,0.65,0.49,0.70,0.67,0.63,0.68,0.71,0.69,0.73,0.81,0.72,0.70


Another research question involved investigating how student satisfaction varied by degree. Firstly, we see how this works *within* LSE, then compare certain courses across universities.

In [71]:
lse_subjectdat=univ_df['LSE']
lse_subjectdat['Subject'].value_counts(dropna=False)

Subject
Law                                                408
Psychology                                         272
Mathematical sciences                              272
Geography, earth and environmental studies         272
Economics                                          272
Politics                                           272
Business and management                            272
Language and area studies                          136
Human geography                                    136
Philosophy                                         136
Philosophy and religious studies                   136
History                                            136
History and archaeology                            136
Historical, philosophical and religious studies    136
Asian studies                                      136
Languages and area studies                         136
Management studies                                 136
Accounting                                         136
Fi

Many of these subjects overlap and so for the following analysis we will ignore certain categories. For example, we will focus on only Psychology instead of Psychology (non-specific) and both Mathematics and Statistics individually rather than 'Mathematical sciences'. We will also remove the missing values since they are connected to the overall 'rating' across all subjects (the metric previously used to compare universities).

### Reddit LSE data

A refresh of the format of the data frame used to store this data:

In [23]:
lse_reddit_data_df = pd.read_csv("Data/reddit_data.csv")
lse_reddit_data_df.head(5)

,Title,Score,Top Comment
0,The irony of LSE being a socialist institution,327,"Don’t really want to give super identifying details but I’m currently a postgraduate student in LSE and although my programme isn’t under the department of economics, I’m in an economics heavy programme. My professors are quite obviously pro-tax (heavily guided by data and evidence of course), pro-government regulations and interventions in the free market, pro-reform and redistribution through public administration and services, and actively teach us how to see through monopolies, inequalities, and undemocratic tactics administered in western free market countries (primarily the US). I think that we may still be considered centrists by hardcore socialists but we are clearly left wing compared to right wing, conservatives, libertarians, and whatever."
1,Got kicked outta LSE.....,238,"Can you share more details? Specifically.\n\nDid you see a DR about such issues and have a record of it?\n\nDid you apply for extenuating circumstances at all?\n\nWas you offered a resit opportunity at all?\n\nWhat is your grounds for appeal, and are you appealing only specifically the final fail decision?"
2,why are masters degrees so expensive?,207,"This doesn't really apply to most masters courses if you are a domestic student, the MFin looks aimed at people who already have professional experience which makes it quite different to most other masters. Even at Cambridge most course are around £12k so almost coverable with the student loan, but still you do need to find £10k+ for living expenses from somewhere"
3,TIL some unis have worse graduate prospects than non-graduates,200,"I think people should bear in mind that Imperial only offers STEM and LSE focuses a lot of financial careers. That inflates the average salaries. In reality, the top 4 are very similar."
4,University subreddits,190,"/r/Edinburgh_University Edinburgh, University of"


The first thing to check is the number of duplicate or null values. As you can see below, there is neither for both.
- There are no null values since the code used to access the top comment from each post was written to only add the post to the dataframe if it contained a comment. 

- There are no duplicate posts since the chance of two identical strings of words is highly unlikely.

In [17]:
num_duplicates = lse_reddit_data_df.duplicated().sum()
print(f"Number of duplicate rows: {num_duplicates}")

Number of duplicate rows: 0


In [18]:
total_nulls = lse_reddit_data_df.isnull().sum().sum()
print(f"Number of null values: {total_nulls}")
print(lse_reddit_data_df.shape)

Number of null values: 0
(30, 3)


We only want to consider posts that have a high amount of upvotes, since they are more likely to be helpful and insightful information if other users liked them. We can acheive this by removing posts whose score is below '30', of which there way only one. This number was decided given the maximum score was 327 and the mean score was 97.

It is worth noting that the posts are already sorted in order of descending score, since that is a paramter included in the Reddit API call.

In [19]:
print(f"Maximum score: {lse_reddit_data_df['Score'].max()}")
print(f"Mean score: {int(lse_reddit_data_df['Score'].mean())}")
print(f"Current number of posts: {lse_reddit_data_df.shape[0]}")
num_low_score_posts = (lse_reddit_data_df['Score']<= 30).sum()
print(f"Number of posts with a score of 5 or below: {num_low_score_posts}")
lse_reddit_data_df = lse_reddit_data_df[lse_reddit_data_df['Score'] > 30]
print(f"New number of posts: {lse_reddit_data_df.shape[0]}")


lse_reddit_data_df.head(5)

Maximum score: 327
Mean score: 97
Current number of posts: 30
Number of posts with a score of 5 or below: 1
New number of posts: 29


,Title,Score,Top Comment
0,The irony of LSE being a socialist institution,327,"Don’t really want to give super identifying details but I’m currently a postgraduate student in LSE and although my programme isn’t under the department of economics, I’m in an economics heavy programme. My professors are quite obviously pro-tax (heavily guided by data and evidence of course), pro-government regulations and interventions in the free market, pro-reform and redistribution through public administration and services, and actively teach us how to see through monopolies, inequalities, and undemocratic tactics administered in western free market countries (primarily the US). I think that we may still be considered centrists by hardcore socialists but we are clearly left wing compared to right wing, conservatives, libertarians, and whatever."
1,Got kicked outta LSE.....,238,"Can you share more details? Specifically.\n\nDid you see a DR about such issues and have a record of it?\n\nDid you apply for extenuating circumstances at all?\n\nWas you offered a resit opportunity at all?\n\nWhat is your grounds for appeal, and are you appealing only specifically the final fail decision?"
2,why are masters degrees so expensive?,207,"This doesn't really apply to most masters courses if you are a domestic student, the MFin looks aimed at people who already have professional experience which makes it quite different to most other masters. Even at Cambridge most course are around £12k so almost coverable with the student loan, but still you do need to find £10k+ for living expenses from somewhere"
3,TIL some unis have worse graduate prospects than non-graduates,200,"I think people should bear in mind that Imperial only offers STEM and LSE focuses a lot of financial careers. That inflates the average salaries. In reality, the top 4 are very similar."
4,University subreddits,190,"/r/Edinburgh_University Edinburgh, University of"


### Reddit COVID-19 Data

Similar to the procedure above, the below dataframe about reddit posts that contain the word 'covid' is imported from a csv file and saved as a dataframe.

In [36]:
covid_reddit_data_df = pd.read_csv("Data/covid_reddit_data.csv")
covid_reddit_data_df.head(5)

,Title,Score,Top Comment
0,Is anyone completely disengaged with university to the point of not attending any lectures/tutorials and only looking at that when it's needed for assignments/exams? I cannot focus with online learning at all and don't even feel like a student,461,[deleted]
1,"Apparently alot of ""covid"" student have been dropping out of Uni",389,"I'm an admissions tutor and lecturer. I teach a fairly challenging module but it's actually quite popular and the final average was about 65%. During Covid we went to online assessment, which resulted in the average shooting up. We returned to a proctored exam last year, and the the average was 38%. This was the 2021 intake (its a Year 2 module). A lot have failed other modules so will be deregistered. Some have quit before they are pushed. Its absolutely not their fault and it's really sad. Their A-levels were severely interrupted, they never learnt how to study properly, and they were totally demoralised. Unfortunately, I think the government will just forget about them. I worry they'll become a lost cohort like the Japanese kids who graduated during the Pacific financial crisis."
2,Important: For people starting uni this year.,383,"Also, not just for you but fellow students, we appreciate there being fewer people to spread whatever arrives on campus."
3,who'd had thunk it,376,Some of us also seem to make this face upon discovering hangovers.
4,Has anyone done a degree they now realise has absolutely no job prospects?,361,What was your undergraduate degree?


To clean the data frame, rows where the value in the 'Top Comment' column was '[deleted]' are removed as this was considered a null value, and where the score is 0.


In [45]:
#remove null values
covid_reddit_data_df = covid_reddit_data_df[covid_reddit_data_df['Top Comment'] != '[deleted]']

#find summary statistics, remove low  scores
print(f"Maximum score: {covid_reddit_data_df['Score'].max()}")
print(f"Mean score: {int(covid_reddit_data_df['Score'].mean())}")
print(f"Current number of posts: {covid_reddit_data_df.shape[0]}")
num_low_score_posts = (covid_reddit_data_df['Score']== 0).sum()
print(f"Number of posts with a score of 0: {num_low_score_posts}")
covid_reddit_data_df = covid_reddit_data_df[covid_reddit_data_df['Score'] > 0]
print(f"New number of posts: {covid_reddit_data_df.shape[0]}")

#display the full comment in the 'Top Comment' column
pd.set_option('display.max_colwidth', None)

covid_reddit_data_df.head(5)

Maximum score: 389
Mean score: 117
Current number of posts: 94
Number of posts with a score of 0: 0
New number of posts: 94


,Title,Score,Top Comment
1,"Apparently alot of ""covid"" student have been dropping out of Uni",389,"I'm an admissions tutor and lecturer. I teach a fairly challenging module but it's actually quite popular and the final average was about 65%. During Covid we went to online assessment, which resulted in the average shooting up. We returned to a proctored exam last year, and the the average was 38%. This was the 2021 intake (its a Year 2 module). A lot have failed other modules so will be deregistered. Some have quit before they are pushed. Its absolutely not their fault and it's really sad. Their A-levels were severely interrupted, they never learnt how to study properly, and they were totally demoralised. Unfortunately, I think the government will just forget about them. I worry they'll become a lost cohort like the Japanese kids who graduated during the Pacific financial crisis."
2,Important: For people starting uni this year.,383,"Also, not just for you but fellow students, we appreciate there being fewer people to spread whatever arrives on campus."
3,who'd had thunk it,376,Some of us also seem to make this face upon discovering hangovers.
4,Has anyone done a degree they now realise has absolutely no job prospects?,361,What was your undergraduate degree?
5,I wasted my fucking time at uni and now I regret ever even going,343,It wasn’t Uni - it was the time and situation. Feel for you mate and hope you sort it out.


The next step was to extract any information about how COVID-19 impacted universtity students in London. The results were as follows:

Only 3/200 posts mentioned the words in the 'keywords' list below that were linked to the city of London, or the names of the biggest universities in London. Of the three posts outputted, only one details the impact of COVID-19 on student life by discussing online learning in the King's College London Physics degree. However, they are descirbing the benefits of this course due to it having compulsory in-person labs, whereas most posts in the previous dataframe have a negative view.

The scores of these posts are also all lower than the mean score of 117, signifying there are not as relevant compared to other posts about COVID-19.

The lack of results could be explained by the following reasons:

- Whether a university was located in London had no impact on the student satisfaction during/after COVID-19.
- Students chose not to mentioned the university they attended when posting on the website.
- The information was not accurately gathered as there are too many variations of university names or other
identifying phrases of a student being located in London.
- The sample size was to small.



In [46]:
keywords = [' lse ', ' ucl ', 'london', ' kcl ', 'kings', ' icl ', 'imperical',
            'imperical', ' ual ', ' uol ', 'university of london ', 'university of west london',
            'university of east london', 'london metropolitan university', ' london met', 'university of greenwhich',
            'university of westminster', 'middlesex university', 'london southbank',
            "regent's university", 'university of roehampton', 'ravensbourne university', 'royal college of art',' rcoa ',
            'birkbeck', ' bpp university', 'guildhall school', 'royal college of music', ' lbs ', 'london business school',
            'rada']

#find posts containing any keywords
london_filtered_df = covid_reddit_data_df[covid_reddit_data_df['Title'].str.contains('|'.join(keywords), case=False, na=False) |
                 covid_reddit_data_df['Top Comment'].str.contains('|'.join(keywords), case=False, na=False)]

london_filtered_df

,Title,Score,Top Comment
48,British but being charged international fees because I spent 4 years studying in the US,100,"I’d say the London university is right. To be considered a home student, you need to be ordinarily resident in the U.K. for the three years before your course starts (e.g. https://www.lse.ac.uk/study-at-lse/international-students/fee-status-classification), which unfortunately you haven’t been, whatever the reason. I’d request a fee status reassessment from both of them to be honest - you wouldn’t want to get half way through the degree at the Scottish university and suddenly have to pay international fees."
62,Questions about clubbing - London,68,"Pryzm is a famously bad club, terrible atmosphere with awful clientele and a hotbed for spiking. What club you want depends on what sort of music you’re into, I club a lot in London but since I love electronic music it’s much more to go to listen to music rather than to try get off with strangers like the likes of Pryzm are for"
71,Terrified Of Online Learning?,51,"It depends what your degree is. Also whether you’re going to be in England or Scotland or NI or Wales because rules can be a bit different. If you’re doing a STEM degree, it’s very likely that you will be on campus (likely some online lectures too) - I received word from KCL that STEM degrees like Physics will have to be on campus because of compulsory labs. If you’re doing something like History or Literature, it will be even more likely you’ll be doing most things online."
